Fazendo a primeira chamada da LLM

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

import pandas as pd
from langchain.docstore.document import Document

Criando nossa primeira chamada de LLM

In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY")
)

In [3]:
llm.invoke("Quem foi Albert Einstein?").content

'Albert Einstein (1879-1955) foi um físico teórico alemão que desenvolveu a teoria da relatividade, um dos dois pilares da física moderna (o outro é a mecânica quântica).  Sua famosa equação, E=mc², que postula a equivalência entre energia e massa, é uma das equações mais conhecidas do mundo.\n\nAlém da relatividade, Einstein fez contribuições significativas em outros campos da física, incluindo a mecânica estatística (explicação do movimento browniano) e a cosmologia (teoria de um universo em expansão).  Ele recebeu o Prêmio Nobel de Física em 1921 por sua explicação do efeito fotoelétrico, que foi crucial para o desenvolvimento da mecânica quântica.\n\nEinstein não foi apenas um gênio científico, mas também uma figura pública altamente influente.  Suas ideias revolucionaram a compreensão do universo e sua imagem icônica se tornou sinônimo de inteligência e genialidade.  Ele também foi um pacifista declarado e um defensor do sionismo.  Sua vida e obra continuam a inspirar e a ser obje

Construindo nossa primeira estrutura de RAG

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
BASE_DIR = Path().resolve()
DOCS_DIR = BASE_DIR / "docs"

Escolhendo a estrutura de embedding

In [6]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\degio\AppData\Local\Temp\ipykernel_23520\323160131.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\degio\Downloads\2025\Preditiva\Formação AI\Conteudos\Módulo 8 - Projetos Finais\CASE 1\Assistente_AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def load_pdf_vectorstore(filepath: str, save_path: str):
    loader = PyPDFLoader(DOCS_DIR / filepath)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
    documents = text_splitter.split_documents(documents)
    vectorstore = FAISS.from_documents(documents, embedding)
    vectorstore.save_local(f"vectorstores/{save_path}")
    retriver = vectorstore.as_retriever(
        search_type="similarity", search_kwargs={"k": 7}
    )
    return retriver

In [8]:
retriver_perguntas_frequentes = load_pdf_vectorstore(
    "Perguntas Frequentes.pdf", "vectorstore_perguntas_frequentes"
)
retriver_manual_tecnico = load_pdf_vectorstore(
    "Manual Tecnico de Produtos.pdf", "vectorstore_manual_tecnico_produtos"
)
retriver_politicas_procedimentos = load_pdf_vectorstore(
    "Politicas e Procedimentos.pdf", "vectorstore_politicas_procedimentos"
)

In [9]:
def load_excel_vectorstore(filepath: str, save_path: str):
    df = pd.read_excel(DOCS_DIR / filepath)
    documents = []
    for idx, row in df.iterrows():
        text = " ".join([str(cell) for cell in row if pd.notna(cell)])
        documents.append(Document(page_content=text, metadata={"row": idx}))

    vectorstore_tickers = FAISS.from_documents(documents, embedding)
    vectorstore_tickers.save_local("vectorstores/vectorstore_tickets")
    retriver_tickets = vectorstore_tickers.as_retriever(
        search_type="similarity", search_kwargs={"k": 7}
    )

    return retriver_tickets

In [10]:
retriver_tickets = load_excel_vectorstore("Tickets.xlsx", "vectorstore_tickets")

Criando os agentes assistentes

In [11]:
from typing import TypedDict, Optional, List
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage

In [12]:
class State(TypedDict, total=False):
    query: str
    route: Optional[str]
    anwser: Optional[str]
    chat_history: Optional[List[BaseMessage]]

In [13]:
def agent_with_retriever(
    state: State, papel: str, prompt_instrucoes: str, retriver=None
):
    query = state["query"]
    chat_history = state.get("chat_history", [])
    contexto = ""

    if retriver:
        recuperados = retriver.get_relevant_documents(query)
        if recuperados:
            contexto = "\n".join([d.page_content for d in recuperados])

    mensagens = [
        SystemMessage(
            content=(
                f"Você é um {papel}. "
                f"Suas instruções:\n{prompt_instrucoes}\n\n"
                f"- Use sempre o contexto recuperado para responder à ÚLTIMA pergunta do usuário.\n"
                f"- Use o histórico da conversa para entender o contexto geral e perguntas de acompanhamento.\n"
                f"- Se não houver informações relevantes no contexto, diga que não encontrou dados suficientes para responder.\n"
                f"- Evite inventar informações."
            )
        ),
        *chat_history,
        HumanMessage(
            content=(
                f"Pergunta do usuário:\n{query}\n\n"
                f"Contexto disponível para esta pergunta:\n{contexto if contexto else '[Nenhum contexto encontrado]'}"
            )
        ),
    ]

    resposta = llm.invoke(mensagens)
    state["anwser"] = resposta.content

    return state

In [14]:
def agent_detalhe_tecnico(state: State):
    prompt_instrucoes = (
        "Seja um **especialista em suporte técnico e produto**. "
        "Você deve responder a perguntas sobre **especificações técnicas**, "
        "**instruções de instalação**, **manutenção preventiva** e **solução de problemas**. "
        "Sua resposta deve ser precisa, técnica e objetiva, baseada estritamente no manual técnico. "
        "Para problemas, ofereça uma solução clara e passo a passo."
    )
    return agent_with_retriever(
        state,
        "especialista em detalhes técnicos",
        prompt_instrucoes,
        retriver_manual_tecnico,
    )

In [15]:
def agent_perguntas_e_respostas(state: State):
    prompt_instrucoes = (
        "Seja um **especialista em Perguntas Frequentes (FAQ)**. "
        "Sua função é fornecer respostas diretas e concisas a perguntas comuns. "
        "Responda como se estivesse consultando uma base de conhecimento, mantendo a resposta factual e sem rodeios. "
        "Se a pergunta se referir a um problema, ofereça a resposta e, se necessário, sugira o contato com o suporte técnico para casos complexos."
    )
    return agent_with_retriever(
        state, "especialista em FAQs", prompt_instrucoes, retriver_perguntas_frequentes
    )

In [16]:
def agent_politicas_e_procedimentos(state: State):
    prompt_instrucoes = (
        "Seja um **especialista em políticas e procedimentos da empresa**. "
        "Sua tarefa é responder a perguntas sobre **garantia**, **horário de atendimento**, "
        "**prazos de SLA** e **regras internas de suporte**. "
        "Sua resposta deve ser formal e baseada nos documentos oficiais, garantindo que o cliente entenda as regras e os processos da empresa."
    )
    return agent_with_retriever(
        state,
        "especialista em políticas e procedimentos",
        prompt_instrucoes,
        retriver_politicas_procedimentos,
    )

In [17]:
def agent_tickets(state: State):
    prompt_instrucoes = (
        "Seja um **especialista em tickets de atendimento**. "
        "Você deve fornecer informações precisas sobre o **status e detalhes de um chamado existente**. "
        "Sua resposta deve ser direta, baseada nos dados do ticket (Ticket ID, Status, Responsável, Descrição do Problema). "
        "Se o usuário perguntar sobre um ticket específico, forneça as informações relevantes e mantenha a resposta curta e direta."
    )
    return agent_with_retriever(
        state,
        "especialista em tickets de atendimento",
        prompt_instrucoes,
        retriver_tickets,
    )

In [18]:
def supervisor(state: State):
    query = state["query"]
    chat_history = state.get("chat_history", [])

    mensagens = [
        SystemMessage(
            content=(
                """Você é um assistente virtual de atendimento ao cliente da **Industech**, uma empresa especializada em produtos industriais.

            Sua principal responsabilidade é atuar como um supervisor, **roteando as perguntas dos clientes para o agente especialista mais adequado**.

            Sua única função é analisar a pergunta do usuário e retornar **uma palavra-chave** que representa o agente responsável. Se a pergunta não se encaixar em nenhuma categoria de especialista, ou se for uma saudação ou uma pergunta sobre suas próprias capacidades, você deve responder de forma amigável diretamente ao cliente, sem rotear.

            A sua resposta deve ser:
            - **Uma frase de resposta direta**, caso a pergunta seja geral (ex: "Olá", "Tudo bem?", "O que você faz?").
            - **Uma das seguintes palavras-chave**, em minúsculas e sem pontuação, para roteamento:
            - **detalhe_tecnico**: para perguntas sobre **especificações técnicas**, **manuais de produtos** ou **solução de problemas**.
            - **perguntas_e_respostas**: para **dúvidas operacionais comuns** ou **FAQs**.
            - **politicas_e_procedimentos**: para questões sobre **políticas da empresa**, **garantia** ou **prazos de atendimento (SLA)**.
            - **tickets**: para perguntas sobre o **status de um chamado**.

            **Regras:**
            - Responda apenas com a frase ou com a palavra-chave.
            - Não adicione explicações, comentários ou qualquer outro texto.
            - Não invente novas categorias."""
            )
        ),
        *chat_history,
        HumanMessage(content=query),
    ]

    resposta = llm.invoke(mensagens)
    resposta_limpa = resposta.content.strip().lower()

    # Verifica se a resposta é uma das rotas ou se é uma resposta direta
    if resposta_limpa in [
        "detalhe_tecnico",
        "perguntas_e_respostas",
        "politicas_e_procedimentos",
        "tickets",
    ]:
        state["route"] = resposta_limpa
    else:
        state["answer"] = resposta.content  # Salva a resposta direta na chave 'answer'

    return state

Workflow

In [19]:
from langgraph.graph import StateGraph, START, END

In [20]:
# Função para decidir o próximo passo
def decide_action(state: State):
    # Se a resposta já foi gerada pelo supervisor, encerra o fluxo.
    if "answer" in state:
        return "end_workflow"
    else:
        # Caso contrário, usa a rota para ir para o agente especialista.
        return state["route"]

In [29]:
def build_workflow():
    workflow = StateGraph(State)

    # Adicionar os nos
    workflow.add_node("supervisor_node", supervisor)
    workflow.add_node("detalhe_tecnico_node", agent_detalhe_tecnico)
    workflow.add_node("perguntas_e_respostas_node", agent_perguntas_e_respostas)
    workflow.add_node("politicas_e_procedimentos_node", agent_politicas_e_procedimentos)
    workflow.add_node("tickets_node", agent_tickets)

    # Adicione um nó de saída para a resposta direta
    workflow.add_node("end_workflow", lambda x: x)

    # Definir o nó inicial
    workflow.add_edge(START, "supervisor_node")

    # Adicione o roteamento condicional
    workflow.add_conditional_edges(
        "supervisor_node",
        decide_action,
        {
            "detalhe_tecnico": "detalhe_tecnico_node",
            "perguntas_e_respostas": "perguntas_e_respostas_node",
            "politicas_e_procedimentos": "politicas_e_procedimentos_node",
            "tickets": "tickets_node",
            "end_workflow": END,  # Termina o fluxo se a resposta já foi gerada pelo supervisor
        },
    )

    # Defina as saídas dos agentes especialistas
    workflow.add_edge("detalhe_tecnico_node", END)
    workflow.add_edge("perguntas_e_respostas_node", END)
    workflow.add_edge("politicas_e_procedimentos_node", END)
    workflow.add_edge("tickets_node", END)

    return workflow.compile()

In [31]:
app = build_workflow()